In [1]:
import PythonicDISORT
from PythonicDISORT.subroutines import _compare
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
from math import pi

import scipy as sc
from scipy import constants

Previously saved reference solutions will be loaded if F2PY-wrapped Stamnes' DISORT is unavailable.

In [2]:
disort_is_installed = True
try:
    import disort
    print("Stamnes' DISORT imported.")
except ImportError:
    disort_is_installed = False
    print("Stamnes' DISORT unavailable. Cached results will be used.")

Stamnes' DISORT imported.


# Table of Contents
* [6b and setup](#6b-and-setup)
* [6c](#6c)
* [6d](#6d)
* [6e](#6e)
* [6f](#6f)
* [6g](#6g)
* [6h](#6h)


# Test Problem 6:  No Scattering, Increasingly Complex Sources (relevant for modeling longwave radiation)

When modeling longwave radiation, scattering is often negligable ($\omega = 0$) but there are many sources that have to be accounted for, not to mention surface reflection. Increasingly complex sources, most of which are relevant for modeling longwave radiation, are added for each subsequent subproblem.

# 6b and setup

The only source is the direct beam. Since there is no scattering, there is only direct radiation which is trivially modeled by Beer's law.

**PythonicDISORT**

In [3]:
######################################### PYDISORT ARGUMENTS #######################################

tau_arr = 1  # One layer of thickness 1 (medium-thick atmosphere)
omega_arr = 0  # No scattering
NQuad = 16  # 16 streams (8 quadrature nodes for each hemisphere)
Leg_coeffs_all = np.zeros(NQuad + 1)
mu0 = 0.5  # Cosine of solar zenith angle (low, glancing angle)
I0 = 200  # Intensity of direct beam
phi0 = 0  # Azimuthal angle of direct beam

# Optional (used)

# Optional (unused)
NLeg = None
NFourier = None
b_pos = 0
b_neg = 0
only_flux = False
f_arr = 0
NT_cor = False
BDRF_Fourier_modes = []
s_poly_coeffs=np.array([[]])
use_banded_solver_NLayers = 10
autograd_compatible=False

####################################################################################################

In [4]:
# Call pydisort function
mu_arr, flux_up, flux_down, u0, u = PythonicDISORT.pydisort(
    tau_arr, omega_arr,
    NQuad,
    Leg_coeffs_all,
    mu0, I0, phi0,
)

**Setup for tests**

In [5]:
# Reorder mu_arr from smallest to largest
reorder_mu = np.argsort(mu_arr)
mu_arr_RO = mu_arr[reorder_mu]

# We may not want to compare intensities around the direct beam
deg_around_beam_to_not_compare = 0
mu_to_compare = (
    np.abs(np.arccos(np.abs(mu_arr_RO)) - np.arccos(mu0)) * 180 / pi
    > deg_around_beam_to_not_compare
)
mu_test_arr_RO = mu_arr_RO[mu_to_compare]

In [6]:
# Number of phi grid points
Nphi = 2
phi_arr = np.random.random(Nphi) * 2 * pi

# tau test points
tau_test_arr = np.array([0, 0.5, 1])
Ntau = len(tau_test_arr)

**Stamnes' DISORT**

In [7]:
# Stamnes' DISORT arguments
nlyr = 1
nmom = NQuad
nstr = NQuad
numu = NQuad
nphi = Nphi
ntau = Ntau
usrang = True
usrtau = True
ibcnd = 0
onlyfl = False
prnt = np.array([False, False, False, False, False])  # Prints to CMD instead of this notebook
plank = False
lamber = False
deltamplus = False
do_pseudo_sphere = False
dtauc = tau_arr
ssalb = omega_arr
pmom = Leg_coeffs_all.T
temper = np.zeros(nlyr + 1)
wvnmlo = 0
wvnmhi = 0
utau = tau_test_arr
umu0 = mu0
phi0 = phi0
umu = mu_arr_RO
phi = phi_arr
fbeam = I0
fisot = 0
albedo = 0
btemp = 0
ttemp = 0
temis = 0
earth_radius = 6371
h_lyr = np.zeros(nlyr + 1)
rhoq = np.zeros((nstr // 2, nstr + 1, nstr))
rhou = np.zeros((numu, nstr // 2 + 1, nstr))
rho_accurate = np.zeros((numu, nphi))
bemst = np.zeros(nstr // 2)
emust = np.zeros(numu)
accur = 0
header = ""
rfldir = np.zeros(ntau)
rfldn = np.zeros(ntau)
flup = np.zeros(ntau)
dfdt = np.zeros(ntau)
uavg = np.zeros(ntau)
uu = np.zeros((numu, ntau, nphi))
albmed = np.zeros(numu)
trnmed = np.zeros(numu)

In [8]:
if disort_is_installed:
    # Run disort, putting DFDT, UAVG, and UU in a, b, and c, respectively
    rfldir, rfldn, flup, dfdt, uavg, uu, albmed, trnmed = disort.disort(usrang, usrtau, ibcnd, onlyfl, prnt, plank, lamber, deltamplus, do_pseudo_sphere, dtauc, ssalb,
                            pmom, temper, wvnmlo, wvnmhi, utau, umu0, phi0 * 180/pi, umu, phi * 180/pi, fbeam, fisot, albedo, btemp, ttemp,
                            temis, earth_radius, h_lyr, rhoq, rhou, rho_accurate, bemst, emust, accur, header, rfldir,
                            rfldn, flup, dfdt, uavg, uu, albmed, trnmed)
    results = {
        "uu": uu,
        "flup": flup,
        "rfldn": rfldn,
        "rfldir": rfldir,
        "tau_test_arr": tau_test_arr,
        "phi_arr": phi_arr
    }
else:
    # Load saved results from Stamnes' DISORT
    results = np.load("Stamnes_results/6b_test.npz")

**Comparisons**

In [9]:
(
    diff_flux_up,
    ratio_flux_up,
    diff_flux_down_diffuse,
    ratio_flux_down_diffuse,
    diff_flux_down_direct,
    ratio_flux_down_direct,
    diff,
    diff_ratio,
) = _compare(results, mu_to_compare, reorder_mu, flux_up, flux_down, u)

Max pointwise differences

Upward (diffuse) fluxes
Difference = 0.0
Difference ratio = 0.0

Downward (diffuse) fluxes
Difference = 0.0
Difference ratio = 0.0

Direct (downward) fluxes
Difference = 6.765569366962154e-07
Difference ratio = 1.8390723931173657e-08

Intensities

At tau = 0.0
Max pointwise difference = 0.0
At tau = 0.0
Max pointwise difference ratio = 0.0



**Does the test pass?**

In [10]:
assert np.max(ratio_flux_up[diff_flux_up > 1e-3], initial=0) < 1e-3
assert np.max(ratio_flux_down_diffuse[diff_flux_down_diffuse > 1e-3], initial=0) < 1e-3
assert np.max(ratio_flux_down_direct[diff_flux_down_direct > 1e-3], initial=0) < 1e-3
assert np.max(diff_ratio[diff > 1e-3], initial=0) < 1e-2

'''
np.savez_compressed (
    "Stamnes_results/6b_test",
    phi_arr=phi_arr,
    tau_test_arr=tau_test_arr,
    uu=uu,
    flup=flup,
    rfldn=rfldn,
    rfldir=rfldir,
)
'''

print("The test passes")

The test passes


-------

# 6c

Direct beam source and Lambertian (isotropic) surface reflection with albedo $\omega_s$.

**PythonicDISORT**

In [11]:
######################################### PYDISORT ARGUMENTS #######################################

tau_arr = 1  # One layer of thickness 1 (medium-thick atmosphere)
omega_arr = 0  # No scattering
NQuad = 16  # 16 streams (8 quadrature nodes for each hemisphere)
Leg_coeffs_all = np.zeros(NQuad + 1)
mu0 = 0.5  # Cosine of solar zenith angle (low, glancing angle)
I0 = 200  # Intensity of direct beam
phi0 = 0  # Azimuthal angle of direct beam

# Optional (used)
omega_s = 0.5
BDRF_Fourier_modes = [lambda mu, neg_mup: np.full((len(mu), len(neg_mup)), omega_s)]

# Optional (unused)
NLeg = None
NFourier = None
b_pos = 0
b_neg = 0
only_flux = False
f_arr = 0
NT_cor = False
s_poly_coeffs=np.array([[]])
use_banded_solver_NLayers = 10
autograd_compatible=False

####################################################################################################

In [12]:
# Call pydisort function
mu_arr, flux_up, flux_down, u0, u = PythonicDISORT.pydisort(
    tau_arr, omega_arr,
    NQuad,
    Leg_coeffs_all,
    mu0, I0, phi0,
    BDRF_Fourier_modes=BDRF_Fourier_modes,
)

**Stamnes' DISORT**

In [13]:
# Stamnes' DISORT arguments
nlyr = 1
nmom = NQuad
nstr = NQuad
numu = NQuad
nphi = Nphi
ntau = Ntau
usrang = True
usrtau = True
ibcnd = 0
onlyfl = False
prnt = np.array([False, False, False, False, False])  # Prints to CMD instead of this notebook
plank = False
lamber = True
deltamplus = False
do_pseudo_sphere = False
dtauc = tau_arr
ssalb = omega_arr
pmom = Leg_coeffs_all.T
temper = np.zeros(nlyr + 1)
wvnmlo = 0
wvnmhi = 0
utau = tau_test_arr
umu0 = mu0
phi0 = phi0
umu = mu_arr_RO
phi = phi_arr
fbeam = I0
fisot = 0
albedo = omega_s
btemp = 0
ttemp = 0
temis = 0
earth_radius = 6371
h_lyr = np.zeros(nlyr + 1)
rhoq = np.zeros((nstr // 2, nstr + 1, nstr))
rhou = np.zeros((numu, nstr // 2 + 1, nstr))
rho_accurate = np.zeros((numu, nphi))
bemst = np.zeros(nstr // 2)
emust = np.zeros(numu)
accur = 0
header = ""
rfldir = np.zeros(ntau)
rfldn = np.zeros(ntau)
flup = np.zeros(ntau)
dfdt = np.zeros(ntau)
uavg = np.zeros(ntau)
uu = np.zeros((numu, ntau, nphi))
albmed = np.zeros(numu)
trnmed = np.zeros(numu)

In [14]:
if disort_is_installed:
    # Run disort, putting DFDT, UAVG, and UU in a, b, and c, respectively
    rfldir, rfldn, flup, dfdt, uavg, uu, albmed, trnmed = disort.disort(usrang, usrtau, ibcnd, onlyfl, prnt, plank, lamber, deltamplus, do_pseudo_sphere, dtauc, ssalb,
                            pmom, temper, wvnmlo, wvnmhi, utau, umu0, phi0 * 180/pi, umu, phi * 180/pi, fbeam, fisot, albedo, btemp, ttemp,
                            temis, earth_radius, h_lyr, rhoq, rhou, rho_accurate, bemst, emust, accur, header, rfldir,
                            rfldn, flup, dfdt, uavg, uu, albmed, trnmed)
    results = {
        "uu": uu,
        "flup": flup,
        "rfldn": rfldn,
        "rfldir": rfldir,
        "tau_test_arr": tau_test_arr,
        "phi_arr": phi_arr
    }
else:
    # Load saved results from `DISOTESTAUX.f` of Stamnes' DISORT
    results = np.load("Stamnes_results/6c_test.npz")

**Comparisons**

In [15]:
(
    diff_flux_up,
    ratio_flux_up,
    diff_flux_down_diffuse,
    ratio_flux_down_diffuse,
    diff_flux_down_direct,
    ratio_flux_down_direct,
    diff,
    diff_ratio,
) = _compare(results, mu_to_compare, reorder_mu, flux_up, flux_down, u)

Max pointwise differences

Upward (diffuse) fluxes
Difference = 1.0844606856963424e-07
Difference ratio = 7.305203278103334e-08

Downward (diffuse) fluxes
Difference = 0.0
Difference ratio = 0.0

Direct (downward) fluxes
Difference = 6.765569366962154e-07
Difference ratio = 1.8390723931173657e-08

Intensities

At tau = 0.5
Max pointwise difference = 1.300719061347877e-07
At tau = 0.0
Max pointwise difference ratio = 2.1190420643433356e-06



**Does the test pass?**

In [16]:
assert np.max(ratio_flux_up[diff_flux_up > 1e-3], initial=0) < 1e-3
assert np.max(ratio_flux_down_diffuse[diff_flux_down_diffuse > 1e-3], initial=0) < 1e-3
assert np.max(ratio_flux_down_direct[diff_flux_down_direct > 1e-3], initial=0) < 1e-3
assert np.max(diff_ratio[diff > 1e-3], initial=0) < 1e-2

'''
np.savez_compressed (
    "Stamnes_results/6c_test",
    phi_arr=phi_arr,
    tau_test_arr=tau_test_arr,
    uu=uu,
    flup=flup,
    rfldn=rfldn,
    rfldir=rfldir,
)
'''

print("The test passes")

The test passes


----------

# 6d

Direct beam source and Hapke (anisotropic) surface reflection (see [[1, p. 233]](#cite-Hap1993) and [[2, p. 44]](#cite-STWLE2000)). Only flux values are compared.

**PythonicDISORT**

In [17]:
from PythonicDISORT.subroutines import cache_BDRF_Fourier_modes

In [18]:
def Hapke(mu, neg_mup, dphi, B0, HH, W):
    cos_alpha = (mu[:, None] * neg_mup[None, :] - np.sqrt(1 - mu**2)[:, None] * np.sqrt(
        (1 - neg_mup**2)[None, :]
    ) * np.cos(dphi)).clip(min=-1, max=1)
    alpha = np.arccos(cos_alpha)

    P = 1 + cos_alpha / 2
    B = B0 * HH / (HH + np.tan(alpha / 2))

    gamma = np.sqrt(1 - W)
    H0 = ((1 + 2 * neg_mup) / (1 + 2 * neg_mup * gamma))[None, :]
    H = ((1 + 2 * mu) / (1 + 2 * mu * gamma))[:, None]

    return W / 4 / (mu[:, None] + neg_mup[None, :]) * ((1 + B) * P + H0 * H - 1)

In [19]:
######################################### PYDISORT ARGUMENTS #######################################

tau_arr = 1  # One layer of thickness 1 (medium-thick atmosphere)
omega_arr = 0  # No scattering
NQuad = 16  # 16 streams (8 quadrature nodes for each hemisphere)
Leg_coeffs_all = np.zeros(NQuad + 1)
mu0 = 0.5  # Cosine of solar zenith angle (low, glancing angle)
I0 = 200  # Intensity of direct beam
phi0 = 0  # Azimuthal angle of direct beam

# Optional (used)
B0, HH, W = 1, 0.06, 0.6
# It is very inefficient to determine the Fourier modes of the BDRF through numerical integration,
# solve for the modes analytically if possible
BDRF_Fourier_modes = [
    lambda mu, neg_mup, m=m: (sc.integrate.quad_vec(
        lambda dphi: Hapke(mu, neg_mup, dphi, B0, HH, W) * np.cos(m * dphi),
        0,
        2 * pi,
    )[0] / ((1 + (m == 0)) * pi))
    for m in range(NQuad)
]
# We cache ``BDRF_Fourier_modes`` since we will re-use them in later tests
cached_BDRF_Fourier_modes = cache_BDRF_Fourier_modes(NQuad // 2, mu0, BDRF_Fourier_modes)
only_flux = True

# Optional (unused)
NLeg = None
NFourier = None
b_pos = 0
b_neg = 0
f_arr = 0
NT_cor = False
s_poly_coeffs = np.array([[]])
use_banded_solver_NLayers = 10
autograd_compatible = False

####################################################################################################


In [20]:
# Call pydisort function
mu_arr, flux_up, flux_down, u0 = PythonicDISORT.pydisort(
    tau_arr, omega_arr,
    NQuad,
    Leg_coeffs_all,
    mu0, I0, phi0,
    BDRF_Fourier_modes=cached_BDRF_Fourier_modes,
    only_flux=only_flux,
)

In [21]:
# Load saved results from `DISOTESTAUX.f` of Stamnes' DISORT
results = np.load("Stamnes_results/6d_test.npz")

**Comparisons**

In [22]:
(
    diff_flux_up,
    ratio_flux_up,
    diff_flux_down_diffuse,
    ratio_flux_down_diffuse,
    diff_flux_down_direct,
    ratio_flux_down_direct,
    #diff,
    #diff_ratio,
) = _compare(results, mu_to_compare, reorder_mu, flux_up, flux_down)

Max pointwise differences

Upward (diffuse) fluxes
Difference = 2.9980446845101483e-06
Difference ratio = 2.155564036488847e-06

Downward (diffuse) fluxes
Difference = 0.0
Difference ratio = 0.0

Direct (downward) fluxes
Difference = 4.411714423468993e-05
Difference ratio = 2.0928556005904317e-06



**Does the test pass?**

In [23]:
assert np.max(ratio_flux_up[diff_flux_up > 1e-3], initial=0) < 1e-3
assert np.max(ratio_flux_down_diffuse[diff_flux_down_diffuse > 1e-3], initial=0) < 1e-3
assert np.max(ratio_flux_down_direct[diff_flux_down_direct > 1e-3], initial=0) < 1e-3
#assert np.max(diff_ratio[diff > 1e-3], initial=0) < 1e-2

print("The test passes")

The test passes


# 6e

Sources: direct beam and blackbody emission from the surface (usually these sources are not used together). There is also Hapke surface reflection and the surface directional emissivity is consistent with the surface reflection in accordance with Kirchoff's law of thermal radiation. Only flux values are compared.

**PythonicDISORT**

In [24]:
from PythonicDISORT.subroutines import generate_emissivity_from_BDRF
from PythonicDISORT.subroutines import blackbody_contrib_to_BCs

In [25]:
def Hapke(mu, neg_mup, dphi, B0, HH, W):
    cos_alpha = (mu[:, None] * neg_mup[None, :] - np.sqrt(1 - mu**2)[:, None] * np.sqrt(
        (1 - neg_mup**2)[None, :]
    ) * np.cos(dphi)).clip(min=-1, max=1)
    alpha = np.arccos(cos_alpha)

    P = 1 + cos_alpha / 2
    B = B0 * HH / (HH + np.tan(alpha / 2))

    gamma = np.sqrt(1 - W)
    H0 = ((1 + 2 * neg_mup) / (1 + 2 * neg_mup * gamma))[None, :]
    H = ((1 + 2 * mu) / (1 + 2 * mu * gamma))[:, None]

    return W / 4 / (mu[:, None] + neg_mup[None, :]) * ((1 + B) * P + H0 * H - 1)

In [26]:
######################################### PYDISORT ARGUMENTS #######################################

tau_arr = 1  # One layer of thickness 1 (medium-thick atmosphere)
omega_arr = 0  # No scattering
NQuad = 16  # 16 streams (8 quadrature nodes for each hemisphere)
Leg_coeffs_all = np.zeros(NQuad + 1)
mu0 = 0.5  # Cosine of solar zenith angle (low, glancing angle)
I0 = 200  # Intensity of direct beam
phi0 = 0  # Azimuthal angle of direct beam

# Optional (used)
B0, HH, W = 1, 0.06, 0.6
BDRF_Fourier_modes = [
    lambda mu, neg_mup, m=m: (sc.integrate.quad_vec(
        lambda dphi: Hapke(mu, neg_mup, dphi, B0, HH, W) * np.cos(m * dphi),
        0,
        2 * pi,
    )[0] / ((1 + (m == 0)) * pi))
    for m in range(NQuad)
] # We will actually use ``cached_BDRF_Fourier_modes`` from test 6d

BTEMP = 300
WVNMLO = 0
WVNMHI = 50000
# The emissivity of the surface should be consistent with the BDRF 
# in accordance with Kirchoff's law of thermal radiation
emissivity = generate_emissivity_from_BDRF(NQuad // 2, BDRF_Fourier_modes[0])
b_pos = emissivity * blackbody_contrib_to_BCs(BTEMP, WVNMLO, WVNMHI)
only_flux = True

# Optional (unused)
NLeg = None
NFourier = None
b_neg = 0
f_arr = 0
NT_cor = False
s_poly_coeffs = np.array([[]])
use_banded_solver_NLayers = 10
autograd_compatible = False

####################################################################################################


In [27]:
# Call pydisort function
mu_arr, flux_up, flux_down, u0 = PythonicDISORT.pydisort(
    tau_arr, omega_arr,
    NQuad,
    Leg_coeffs_all,
    mu0, I0, phi0,
    b_pos=b_pos,
    BDRF_Fourier_modes=cached_BDRF_Fourier_modes,
    only_flux=only_flux,
)

In [28]:
# Load saved results from `DISOTESTAUX.f` of Stamnes' DISORT
results = np.load("Stamnes_results/6e_test.npz")

**Comparisons**

In [29]:
(
    diff_flux_up,
    ratio_flux_up,
    diff_flux_down_diffuse,
    ratio_flux_down_diffuse,
    diff_flux_down_direct,
    ratio_flux_down_direct,
    #diff,
    #diff_ratio,
) = _compare(results, mu_to_compare, reorder_mu, flux_up, flux_down)

Max pointwise differences

Upward (diffuse) fluxes
Difference = 0.07146771916177386
Difference ratio = 0.00022678643977689422

Downward (diffuse) fluxes
Difference = 0.0
Difference ratio = 0.0

Direct (downward) fluxes
Difference = 4.411714423468993e-05
Difference ratio = 2.0928556005904317e-06



**Does the test pass?**

In [30]:
assert np.max(ratio_flux_up[diff_flux_up > 1e-3], initial=0) < 1e-3
assert np.max(ratio_flux_down_diffuse[diff_flux_down_diffuse > 1e-3], initial=0) < 1e-3
assert np.max(ratio_flux_down_direct[diff_flux_down_direct > 1e-3], initial=0) < 1e-3
#assert np.max(diff_ratio[diff > 1e-3], initial=0) < 1e-2

print("The test passes")

The test passes


-------

# 6f

Sources: direct beam, blackbody emission from the surface, blackbody emission plus additional isotropic radiation from the top boundary. There is also Hapke surface reflection and the surface directional emissivity is consistent with the surface reflection in accordance with Kirchoff's law of thermal radiation. The emissivity of the top boundary is set to $1$. Only flux values are compared.

**PythonicDISORT**

In [31]:
from PythonicDISORT.subroutines import generate_emissivity_from_BDRF
from PythonicDISORT.subroutines import blackbody_contrib_to_BCs

In [32]:
def Hapke(mu, neg_mup, dphi, B0, HH, W):
    cos_alpha = (mu[:, None] * neg_mup[None, :] - np.sqrt(1 - mu**2)[:, None] * np.sqrt(
        (1 - neg_mup**2)[None, :]
    ) * np.cos(dphi)).clip(min=-1, max=1)
    alpha = np.arccos(cos_alpha)

    P = 1 + cos_alpha / 2
    B = B0 * HH / (HH + np.tan(alpha / 2))

    gamma = np.sqrt(1 - W)
    H0 = ((1 + 2 * neg_mup) / (1 + 2 * neg_mup * gamma))[None, :]
    H = ((1 + 2 * mu) / (1 + 2 * mu * gamma))[:, None]

    return W / 4 / (mu[:, None] + neg_mup[None, :]) * ((1 + B) * P + H0 * H - 1)

In [33]:
######################################### PYDISORT ARGUMENTS #######################################

tau_arr = 1  # One layer of thickness 1 (medium-thick atmosphere)
omega_arr = 0  # No scattering
NQuad = 16  # 16 streams (8 quadrature nodes for each hemisphere)
Leg_coeffs_all = np.zeros(NQuad + 1)
mu0 = 0.5  # Cosine of solar zenith angle (low, glancing angle)
I0 = 200  # Intensity of direct beam
phi0 = 0  # Azimuthal angle of direct beam

# Optional (used)
B0, HH, W = 1, 0.06, 0.6
BDRF_Fourier_modes = [
    lambda mu, neg_mup, m=m: (sc.integrate.quad_vec(
        lambda dphi: Hapke(mu, neg_mup, dphi, B0, HH, W) * np.cos(m * dphi),
        0,
        2 * pi,
    )[0] / ((1 + (m == 0)) * pi))
    for m in range(NQuad)
] # We will actually use ``cached_BDRF_Fourier_modes`` from test 6d

BTEMP = 300
TTEMP = 250
WVNMLO = 0
WVNMHI = 50000
# The emissivity of the surface should be consistent with the BDRF 
# in accordance with Kirchoff's law of thermal radiation
emissivity = generate_emissivity_from_BDRF(NQuad // 2, BDRF_Fourier_modes[0])
b_pos = emissivity * blackbody_contrib_to_BCs(BTEMP, WVNMLO, WVNMHI) 
b_neg = blackbody_contrib_to_BCs(TTEMP, WVNMLO, WVNMHI) + 100 / pi # Emissivity 1
only_flux = True

# Optional (unused)
NLeg = None
NFourier = None
f_arr = 0
NT_cor = False
s_poly_coeffs = np.array([[]])
use_banded_solver_NLayers = 10
autograd_compatible = False

####################################################################################################


In [34]:
# Call pydisort function
mu_arr, flux_up, flux_down, u0 = PythonicDISORT.pydisort(
    tau_arr, omega_arr,
    NQuad,
    Leg_coeffs_all,
    mu0, I0, phi0,
    b_pos=b_pos,
    b_neg=b_neg,
    BDRF_Fourier_modes=cached_BDRF_Fourier_modes,
    only_flux=only_flux,
)

In [35]:
# Load saved results from `DISOTESTAUX.f` of Stamnes' DISORT
results = np.load("Stamnes_results/6f_test.npz")

**Comparisons**

In [36]:
(
    diff_flux_up,
    ratio_flux_up,
    diff_flux_down_diffuse,
    ratio_flux_down_diffuse,
    diff_flux_down_direct,
    ratio_flux_down_direct,
    #diff,
    #diff_ratio,
) = _compare(results, mu_to_compare, reorder_mu, flux_up, flux_down)

Max pointwise differences

Upward (diffuse) fluxes
Difference = 0.07142052789885156
Difference ratio = 0.00021798172634933044

Downward (diffuse) fluxes
Difference = 0.0020007493918683394
Difference ratio = 7.137215957925398e-06

Direct (downward) fluxes
Difference = 4.411714423468993e-05
Difference ratio = 2.0928556005904317e-06



**Does the test pass?**

In [37]:
assert np.max(ratio_flux_up[diff_flux_up > 1e-3], initial=0) < 1e-3
assert np.max(ratio_flux_down_diffuse[diff_flux_down_diffuse > 1e-3], initial=0) < 1e-3
assert np.max(ratio_flux_down_direct[diff_flux_down_direct > 1e-3], initial=0) < 1e-3
#assert np.max(diff_ratio[diff > 1e-3], initial=0) < 1e-2

print("The test passes")

The test passes


--------

# 6g

Sources: direct beam, blackbody emission from the surface, blackbody emission plus additional isotropic radiation from the top boundary, internal blackbody emission. There is also Hapke surface reflection and the surface directional emissivity is consistent with the surface reflection in accordance with Kirchoff's law of thermal radiation. The emissivities of other blackbodies are set to $1$. Only flux values are compared.

**PythonicDISORT**

In [38]:
from PythonicDISORT.subroutines import generate_emissivity_from_BDRF
from PythonicDISORT.subroutines import blackbody_contrib_to_BCs
from PythonicDISORT.subroutines import generate_s_poly_coeffs

In [39]:
def Hapke(mu, neg_mup, dphi, B0, HH, W):
    cos_alpha = (mu[:, None] * neg_mup[None, :] - np.sqrt(1 - mu**2)[:, None] * np.sqrt(
        (1 - neg_mup**2)[None, :]
    ) * np.cos(dphi)).clip(min=-1, max=1)
    alpha = np.arccos(cos_alpha)

    P = 1 + cos_alpha / 2
    B = B0 * HH / (HH + np.tan(alpha / 2))

    gamma = np.sqrt(1 - W)
    H0 = ((1 + 2 * neg_mup) / (1 + 2 * neg_mup * gamma))[None, :]
    H = ((1 + 2 * mu) / (1 + 2 * mu * gamma))[:, None]

    return W / 4 / (mu[:, None] + neg_mup[None, :]) * ((1 + B) * P + H0 * H - 1)

In [40]:
######################################### PYDISORT ARGUMENTS #######################################

tau_arr = 1  # One layer of thickness 1 (medium-thick atmosphere)
omega_arr = 0  # No scattering
NQuad = 16  # 16 streams (8 quadrature nodes for each hemisphere)
Leg_coeffs_all = np.zeros(NQuad + 1)
mu0 = 0.5  # Cosine of solar zenith angle (low, glancing angle)
I0 = 200  # Intensity of direct beam
phi0 = 0  # Azimuthal angle of direct beam

# Optional (used)
B0, HH, W = 1, 0.06, 0.6
BDRF_Fourier_modes = [
    lambda mu, neg_mup, m=m: (sc.integrate.quad_vec(
        lambda dphi: Hapke(mu, neg_mup, dphi, B0, HH, W) * np.cos(m * dphi),
        0,
        2 * pi,
    )[0] / ((1 + (m == 0)) * pi))
    for m in range(NQuad)
] # We will actually use ``cached_BDRF_Fourier_modes`` from test 6d

BTEMP = 300
TTEMP = 250
TEMPER = np.array([250, 300])
WVNMLO = 0
WVNMHI = 50000
# The emissivity of the surface should be consistent with the BDRF 
# in accordance with Kirchoff's law of thermal radiation
emissivity = generate_emissivity_from_BDRF(NQuad // 2, BDRF_Fourier_modes[0])
b_pos = emissivity * blackbody_contrib_to_BCs(BTEMP, WVNMLO, WVNMHI) 
b_neg = blackbody_contrib_to_BCs(TTEMP, WVNMLO, WVNMHI) + 100 / pi # Emissivity 1
s_poly_coeffs = generate_s_poly_coeffs(tau_arr, TEMPER, WVNMLO, WVNMHI) # Emissivity 1
only_flux = True

# Optional (unused)
NLeg = None
NFourier = None
f_arr = 0
NT_cor = False
use_banded_solver_NLayers = 10
autograd_compatible = False

####################################################################################################


In [41]:
# Call pydisort function
mu_arr, flux_up, flux_down, u0 = PythonicDISORT.pydisort(
    tau_arr, omega_arr,
    NQuad,
    Leg_coeffs_all,
    mu0, I0, phi0,
    b_pos=b_pos,
    b_neg=b_neg,
    BDRF_Fourier_modes=cached_BDRF_Fourier_modes,
    s_poly_coeffs=s_poly_coeffs,
    only_flux=only_flux,
)

In [42]:
# Load saved results from `DISOTESTAUX.f` of Stamnes' DISORT
results = np.load("Stamnes_results/6g_test.npz")

**Comparisons**

In [43]:
(
    diff_flux_up,
    ratio_flux_up,
    diff_flux_down_diffuse,
    ratio_flux_down_diffuse,
    diff_flux_down_direct,
    ratio_flux_down_direct,
    #diff,
    #diff_ratio,
) = _compare(results, mu_to_compare, reorder_mu, flux_up, flux_down)

Max pointwise differences

Upward (diffuse) fluxes
Difference = 0.07081099427045956
Difference ratio = 0.0001605236481053206

Downward (diffuse) fluxes
Difference = 0.0037395118030758567
Difference ratio = 1.0283780863828972e-05

Direct (downward) fluxes
Difference = 4.411714423468993e-05
Difference ratio = 2.0928556005904317e-06



**Does the test pass?**

In [44]:
assert np.max(ratio_flux_up[diff_flux_up > 1e-3], initial=0) < 1e-3
assert np.max(ratio_flux_down_diffuse[diff_flux_down_diffuse > 1e-3], initial=0) < 1e-3
assert np.max(ratio_flux_down_direct[diff_flux_down_direct > 1e-3], initial=0) < 1e-3
#assert np.max(diff_ratio[diff > 1e-3], initial=0) < 1e-2

print("The test passes")

The test passes


--------

# 6h

Same as test 6g but the optical depth is increased from $1$ to $10$.

**PythonicDISORT**

In [45]:
from PythonicDISORT.subroutines import generate_emissivity_from_BDRF
from PythonicDISORT.subroutines import blackbody_contrib_to_BCs
from PythonicDISORT.subroutines import generate_s_poly_coeffs

In [46]:
def Hapke(mu, neg_mup, dphi, B0, HH, W):
    cos_alpha = (mu[:, None] * neg_mup[None, :] - np.sqrt(1 - mu**2)[:, None] * np.sqrt(
        (1 - neg_mup**2)[None, :]
    ) * np.cos(dphi)).clip(min=-1, max=1)
    alpha = np.arccos(cos_alpha)

    P = 1 + cos_alpha / 2
    B = B0 * HH / (HH + np.tan(alpha / 2))

    gamma = np.sqrt(1 - W)
    H0 = ((1 + 2 * neg_mup) / (1 + 2 * neg_mup * gamma))[None, :]
    H = ((1 + 2 * mu) / (1 + 2 * mu * gamma))[:, None]

    return W / 4 / (mu[:, None] + neg_mup[None, :]) * ((1 + B) * P + H0 * H - 1)

In [47]:
######################################### PYDISORT ARGUMENTS #######################################

tau_arr = 10  # One layer of thickness 1 (thick atmosphere)
omega_arr = 0  # No scattering
NQuad = 16  # 16 streams (8 quadrature nodes for each hemisphere)
Leg_coeffs_all = np.zeros(NQuad + 1)
mu0 = 0.5  # Cosine of solar zenith angle (low, glancing angle)
I0 = 200  # Intensity of direct beam
phi0 = 0  # Azimuthal angle of direct beam

# Optional (used)
B0, HH, W = 1, 0.06, 0.6
BDRF_Fourier_modes = [
    lambda mu, neg_mup, m=m: (sc.integrate.quad_vec(
        lambda dphi: Hapke(mu, neg_mup, dphi, B0, HH, W) * np.cos(m * dphi),
        0,
        2 * pi,
    )[0] / ((1 + (m == 0)) * pi))
    for m in range(NQuad)
] # We will actually use ``cached_BDRF_Fourier_modes`` from test 6d

BTEMP = 300
TTEMP = 250
TEMPER = np.array([250, 300])
WVNMLO = 0
WVNMHI = 50000
# The emissivity of the surface should be consistent with the BDRF 
# in accordance with Kirchoff's law of thermal radiation
emissivity = generate_emissivity_from_BDRF(NQuad // 2, BDRF_Fourier_modes[0])
b_pos = emissivity * blackbody_contrib_to_BCs(BTEMP, WVNMLO, WVNMHI) 
b_neg = blackbody_contrib_to_BCs(TTEMP, WVNMLO, WVNMHI) + 100 / pi # Emissivity 1
s_poly_coeffs = generate_s_poly_coeffs(tau_arr, TEMPER, WVNMLO, WVNMHI) # Emissivity 1
only_flux = True

# Optional (unused)
NLeg = None
NFourier = None
f_arr = 0
NT_cor = False
use_banded_solver_NLayers = 10
autograd_compatible = False

####################################################################################################


In [48]:
# Call pydisort function
mu_arr, flux_up, flux_down, u0 = PythonicDISORT.pydisort(
    tau_arr, omega_arr,
    NQuad,
    Leg_coeffs_all,
    mu0, I0, phi0,
    b_pos=b_pos,
    b_neg=b_neg,
    BDRF_Fourier_modes=cached_BDRF_Fourier_modes,
    s_poly_coeffs=s_poly_coeffs,
    only_flux=only_flux,
)

In [49]:
# Load saved results from `DISOTESTAUX.f` of Stamnes' DISORT
results = np.load("Stamnes_results/6h_test.npz")

**Comparisons**

In [50]:
(
    diff_flux_up,
    ratio_flux_up,
    diff_flux_down_diffuse,
    ratio_flux_down_diffuse,
    diff_flux_down_direct,
    ratio_flux_down_direct,
    #diff,
    #diff_ratio,
) = _compare(results, mu_to_compare, reorder_mu, flux_up, flux_down)

Max pointwise differences

Upward (diffuse) fluxes
Difference = 0.07012573446155557
Difference ratio = 0.0001538313969862646

Downward (diffuse) fluxes
Difference = 0.003773035576273287
Difference ratio = 8.508482641039877e-06

Direct (downward) fluxes
Difference = 2.8323661270590605e-05
Difference ratio = 2.0928556005904317e-06



**Does the test pass?**

In [51]:
assert np.max(ratio_flux_up[diff_flux_up > 1e-3], initial=0) < 1e-3
assert np.max(ratio_flux_down_diffuse[diff_flux_down_diffuse > 1e-3], initial=0) < 1e-3
assert np.max(ratio_flux_down_direct[diff_flux_down_direct > 1e-3], initial=0) < 1e-3
#assert np.max(diff_ratio[diff > 1e-3], initial=0) < 1e-2

print("The test passes")

The test passes


--------

<!--bibtex

@article{STWLE2000,
author = {Tsay, Si-Chee and Stamnes, Knut and Wiscombe, Warren and Laszlo, Istvan and Einaudi, Franco},
year = {2000},
month = {02},
pages = {},
title = {General Purpose Fortran Program for Discrete-Ordinate-Method Radiative Transfer in Scattering and Emitting Layered Media: An Update of DISORT}
}

@inbook{Hap1993, place={Cambridge}, series={Topics in Remote Sensing}, title={The bidirectional reflectance of a semiinfinite medium}, booktitle={Theory of Reflectance and Emittance Spectroscopy}, publisher={Cambridge University Press}, author={Hapke, Bruce}, year={1993}, pages={181–235}, collection={Topics in Remote Sensing}}

-->

# References

<a id="cite-Hap1993"/><sup><a href=#ref-1>[^]</a></sup>Hapke, Bruce. 1993. _The bidirectional reflectance of a semiinfinite medium_.

<a id="cite-STWLE2000"/><sup><a href=#ref-2>[^]</a></sup>Tsay, Si-Chee and Stamnes, Knut and Wiscombe, Warren and Laszlo, Istvan and Einaudi, Franco. 2000. _General Purpose Fortran Program for Discrete-Ordinate-Method Radiative Transfer in Scattering and Emitting Layered Media: An Update of DISORT_.

